Import Pandas package so we can use Pandas dataframes and json package so we can use json files

In [58]:
import pandas as pd
import json

Function to load the .csv files containing the static tables. This function takes a list as input where each list item is the string of a pathway to a .csv file. The files are then opened as Pandas dataframes and stored in a dictionary where the filename is the key and the dataframe is the value.

In [59]:
def Load_csvs(list):
    dic_of_dfs = {}
    for i in list:
        dfname = i.rsplit('\\', 1)[-1]
        dic_of_dfs[dfname] = pd.read_csv(i)
    return dic_of_dfs

In [60]:
list_of_pathways_to_csvs = [#insert list of pathways to csv files here]

Function to create a json containing the user input values. The input values in this function are merely standins until actual user input is provided, at which point the json file will be updated.

In [61]:
def Create_Input_Json():
    inputs = {'General Site Data': {'Site Name': 'Dutch Example',
                                     'Total Area': 1, 
                                     'Coordinates': [0, 0], 
                                     'Elevation': 0, 
                                     'Peat Type': 'Sphagnum',
                                     'Peat Thickness': 100,
                                     'Year Rewetting Started': 2022}, 
            'Baseline': {'Median Groundwater Level in Summer': -10,
                         'Vegetation Class': 'G5s:Wet Grassland with Shunt Species',
                         'Animal Maunre Applied': 150,
                         'Organic Fertilizer Applied': 0,
                         'Synthetic Fertilizer': 'Nitrate Based',
                         'Amount Applied': 0,
                         'Grazing Animals': 'No Animal',
                         'Average Number of Grazing Animals': 0,
                         'Average Number of Grazing Days per Year': 0,
                         'Crop Yield': 12,
                         'Crop Residue': 'No',
                         'Crop': 'Cattail (Typha Sp.)',
                         'Litres of Diesel per Site': 0,
                         'Electricity per Site': 0},
            'Rewetting': {'Median Groundwater Level in Summer': -10,
                         'Vegetation Class': 'G5s:Wet Grassland with Shunt Species',
                         'Animal Maunre Applied': 150,
                         'Organic Fertilizer Applied': 0,
                         'Synthetic Fertilizer': 'Nitrate Based',
                         'Amount Applied': 0,
                         'Grazing Animals': 'No Animal',
                         'Average Number of Grazing Animals': 0,
                         'Average Number of Grazing Days per Year': 0,
                         'Crop Yield': 12,
                         'Crop Residue': 'No',
                         'Crop': 'Cattail (Typha Sp.)',
                         'Litres of Diesel per Site': 0,
                         'Electricity per Site': 0,
                         'Crop Use': 'Paper'}}
    
    #inputs = json.dumps(inputs, indent = 1)
    
    with open('user_input_SET.json', 'w') as outfile:
        print(json.dumps(inputs, indent = 5), file = outfile)

Create the Data tab by calling information from the Input json and GEST 2.0 dataframe. This function takes two parameters. The first is a string of the name of the json file where the user input data is stored, the second is the dataframe which contains the GEST 2.0 table. This function returns a pandas dataframe which contains the output provided in the data tab of the excel file.

In [62]:
def Create_Data_Tab(user_input, gest):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initiate the Data dict
    data = {'Baseline': {}, 'Rewetting': {}}

    #Start populating the data dict with simple data
    data['Baseline']['Vegetation Class'] = user_input['Baseline']['Vegetation Class']
    data['Baseline']['Grazing Animals'] = user_input['Baseline']['Grazing Animals']
    data['Baseline']['Synthetic Fertilizer'] = user_input['Baseline']['Synthetic Fertilizer']

    data['Rewetting']['Vegetation Class'] = user_input['Rewetting']['Vegetation Class']
    data['Rewetting']['Grazing Animals'] = user_input['Rewetting']['Grazing Animals']
    data['Rewetting']['Synthetic Fertilizer'] = user_input['Rewetting']['Synthetic Fertilizer']

    #Calculate the animal emission factor and nitrogen excretion per head per year
    if data['Baseline']['Grazing Animals'] == 'No Animals':
        data['Baseline']['Animal Emission Factor'] = 0
        data['Baseline']['Nitrogen Excretion'] = 0
    elif data['Baseline']['Grazing Animals'] == 'Dairy Cattle':
        data['Baseline']['Animal Emission Factor'] = 0.02
        data['Baseline']['Nitrogen Excretion'] = 130.1
    elif data['Baseline']['Grazing Animals'] == 'Beef Cattle':
        data['Baseline']['Animal Emission Factor'] = 0.02
        data['Baseline']['Nitrogen Excretion'] = 28.2
    elif data['Baseline']['Grazing Animals'] == 'Sheep':
        data['Baseline']['Animal Emission Factor'] = 0.01
        data['Baseline']['Nitrogen Excretion'] = 9.9
    elif data['Baseline']['Grazing Animals'] == 'Goats':
        data['Baseline']['Animal Emission Factor'] = 0.01
        data['Baseline']['Nitrogen Excretion'] = 7.4
    elif data['Baseline']['Grazing Animals'] == 'Water Buffalo':
        data['Baseline']['Animal Emission Factor'] = 0.02
        data['Baseline']['Nitrogen Excretion'] = 76.5

    if data['Rewetting']['Grazing Animals'] == 'No Animals':
        data['Rewetting']['Animal Emission Factor'] = 0
        data['Rewetting']['Nitrogen Excretion'] = 0
    elif data['Rewetting']['Grazing Animals'] == 'Dairy Cattle':
        data['Rewetting']['Animal Emission Factor'] = 0.02
        data['Rewetting']['Nitrogen Excretion'] = 130.1
    elif data['Rewetting']['Grazing Animals'] == 'Beef Cattle':
        data['Rewetting']['Animal Emission Factor'] = 0.02
        data['Rewetting']['Nitrogen Excretion'] = 28.2
    elif data['Rewetting']['Grazing Animals'] == 'Sheep':
        data['Rewetting']['Animal Emission Factor'] = 0.01
        data['Rewetting']['Nitrogen Excretion'] = 9.9
    elif data['Rewetting']['Grazing Animals'] == 'Goats':
        data['Rewetting']['Animal Emission Factor'] = 0.01
        data['Rewetting']['Nitrogen Excretion'] = 7.4
    elif data['Rewetting']['Grazing Animals'] == 'Water Buffalo':
        data['Rewetting']['Animal Emission Factor'] = 0.02
        data['Rewetting']['Nitrogen Excretion'] = 76.5

    #Calculate synthetic fertilizer emission factor
    if data['Baseline']['Synthetic Fertilizer'] == 'Nitrate Based':
        data['Baseline']['Fertilizer Emission Factor'] = 0.015
    elif data['Baseline']['Synthetic Fertilizer'] == 'Ammonium Based':
        data['Baseline']['Fertilizer Emission Factor'] = 0.01

    if data['Rewetting']['Synthetic Fertilizer'] == 'Nitrate Based':
        data['Rewetting']['Fertilizer Emission Factor'] = 0.015
    elif data['Rewetting']['Synthetic Fertilizer'] == 'Ammonium Based':
        data['Rewetting']['Fertilizer Emission Factor'] = 0.01

    #Find Crop Residue number
    if user_input['Baseline']['Crop Residue'] == 'Yes':
        data['Baseline']['Crop Residue'] = 1
    else:
        data['Baseline']['Crop Residue'] = 2

    if user_input['Rewetting']['Crop Residue'] == 'Yes':
        data['Rewetting']['Crop Residue'] = 1
    else:
        data['Rewetting']['Crop Residue'] = 2

    #Find Number of Crop Name
    if user_input['Baseline']['Crop'] == 'Cattail (Typha Sp.)':
        data['Baseline']['Crop'] = 1
    elif user_input['Baseline']['Crop'] == 'Reed (Phragmites Australis)':
        data['Baseline']['Crop'] = 2
    elif user_input['Baseline']['Crop'] == 'Peat Moss (Sphagnum Sp.) ':
        data['Baseline']['Crop'] = 3
    elif user_input['Baseline']['Crop'] == 'Grasses like Reed Canary Grass':
        data['Baseline']['Crop'] = 4
    elif user_input['Baseline']['Crop'] == 'Alder (Alnus Sp.)':
        data['Baseline']['Crop'] = 5
    elif user_input['Baseline']['Crop'] == 'Other':
        data['Baseline']['Crop'] = 6

    if user_input['Rewetting']['Crop'] == 'Cattail (Typha Sp.)':
        data['Rewetting']['Crop'] = 1
    elif user_input['Rewetting']['Crop'] == 'Reed (Phragmites Australis)':
        data['Rewetting']['Crop'] = 2
    elif user_input['Rewetting']['Crop'] == 'Peat Moss (Sphagnum Sp.) ':
        data['Rewetting']['Crop'] = 3
    elif user_input['Rewetting']['Crop'] == 'Grasses like Reed Canary Grass':
        data['Rewetting']['Crop'] = 4
    elif user_input['Rewetting']['Crop'] == 'Alder (Alnus Sp.)':
        data['Rewetting']['Crop'] = 5
    elif user_input['Rewetting']['Crop'] == 'Other':
        data['Rewetting']['Crop'] = 6

    #Find Product Use Number
    if user_input['Rewetting']['Crop Use'] == 'Building Materials e.g insulation, taching, timber':
        data['Rewetting']['Crop Use'] = 1
    elif user_input['Rewetting']['Crop Use'] == 'Bedding Material':
        data['Rewetting']['Crop Use'] = 2
    elif user_input['Rewetting']['Crop Use'] == 'Food Application':
        data['Rewetting']['Crop Use'] = 3
    elif user_input['Rewetting']['Crop Use'] == 'Fodder/Feed Application':
        data['Rewetting']['Crop Use'] = 4
    elif user_input['Rewetting']['Crop Use'] == 'Energy Use: biogas, combustion, wood etc':
        data['Rewetting']['Crop Use'] = 5
    elif user_input['Rewetting']['Crop Use'] == 'Paper':
        data['Rewetting']['Crop Use'] = 6
    elif user_input['Rewetting']['Crop Use'] == 'Extraction of Ingredients/Building Blocks: proteins, fibres, cellulose etc':
        data['Rewetting']['Crop Use'] = 7
    elif user_input['Rewetting']['Crop Use'] == 'High Quality Substrate in Horticulture':
        data['Rewetting']['Crop Use'] = 8
    elif user_input['Rewetting']['Crop Use'] == 'Other Uses/Unknown':
        data['Rewetting']['Crop Use'] = 9

    #Perform vegetation class emission calculations
    veg_num_base = data['Baseline']['Vegetation Class'].split(':', 1)[0]
    veg_num_rewet = data['Rewetting']['Vegetation Class'].split(':', 1)[0]
    
    for i in range(2, 44):
        if veg_num_base == gest.iloc[i]['Name']:
            data['Baseline']['Vegetation CH4 GWP'] = gest.iloc[i]['CH4']
            data['Baseline']['Vegetation CO2 GWP'] = gest.iloc[i]['CO2']
            data['Baseline']['Vegetation Total C-Flux GWP'] = gest.iloc[i]['Total C-flux']
        if veg_num_rewet == gest.iloc[i]['Name']:
            data['Rewetting']['Vegetation CH4 GWP'] = gest.iloc[i]['CH4']
            data['Rewetting']['Vegetation CO2 GWP'] = gest.iloc[i]['CO2']
            data['Rewetting']['Vegetation Total C-Flux GWP'] = gest.iloc[i]['Total C-flux']

    return pd.DataFrame.from_dict(data)
    


In [63]:
dict_of_csvs = Load_csvs(list_of_pathways_to_csvs)
gest = dict_of_csvs['GEST_2_Static_Values.csv']

In [64]:
Create_Data_Tab('user_input_SET.json', gest)

,Baseline,Rewetting
Vegetation Class,G5s:Wet Grassland with Shunt Species,G5s:Wet Grassland with Shunt Species
Grazing Animals,No Animal,No Animal
Synthetic Fertilizer,Nitrate Based,Nitrate Based
Fertilizer Emission Factor,0.015,0.015
Crop Residue,2,2
Crop,1,1
Vegetation CH4 GWP,2.93,2.93
Vegetation CO2 GWP,-3.89,-3.89
Vegetation Total C-Flux GWP,-1,-1
Crop Use,NaN,6
